# Checkout Flow Optimization: Data Exploration

**Notebook:** exploration  
**Purpose:** Initial exploration of simulated checkout funnel data  
**Data Source:** DuckDB warehouse (`duckdb/warehouse.duckdb`)  

---

## Notebook Goals

1. **Verify event volumes** by date and variant
2. **Inspect funnel step-through rates** across checkout stages
3. **Preview guardrails** (payment authorization rate, order value distribution)
4. **Produce visualizations:**
   - Funnel bar chart showing conversion at each stage
   - Latency distribution across checkout steps

---


> **Note:** Data are synthetic, generated by `src/data/simulate.py`, stored as Parquet in `data/raw/`, queried via DuckDB views in `sql/schema.sql` and marts in `sql/marts/*`. This is for exploration only.


## Setup & Imports


In [ ]:
import duckdb
import os
from pathlib import Path

# Ensure we're working from project root
project_root = Path(__file__).resolve().parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_root)

# Connect to warehouse
conn = duckdb.connect("duckdb/warehouse.duckdb")

# Print DuckDB version
version = conn.execute("SELECT version()").fetchone()[0]
print(f"DuckDB Version: {version}")
print()

# Get most recent date and row count
result = conn.execute("""
    SELECT 
        MAX(date) as max_date,
        COUNT(*) as row_count
    FROM events.add_to_cart
    WHERE date = (SELECT MAX(date) FROM events.add_to_cart)
""").fetchone()

max_date, row_count = result
print(f"Most Recent Date: {max_date}")
print(f"events.add_to_cart row count for {max_date}: {row_count:,}")

conn.close()


## 1. Event Volume Verification

Verify data completeness and balance across:
- Date range coverage
- Event counts per date
- Variant distribution (control vs treatment)


### 1.1 Event Counts by Date


In [ ]:
# Query event counts by date across all six events
event_counts_by_date = conn.execute("""
    SELECT 
        date,
        COALESCE(SUM(CASE WHEN event_type = 'add_to_cart' THEN cnt END), 0) as add_to_cart,
        COALESCE(SUM(CASE WHEN event_type = 'begin_checkout' THEN cnt END), 0) as begin_checkout,
        COALESCE(SUM(CASE WHEN event_type = 'checkout_step_view' THEN cnt END), 0) as checkout_step_view,
        COALESCE(SUM(CASE WHEN event_type = 'form_error' THEN cnt END), 0) as form_error,
        COALESCE(SUM(CASE WHEN event_type = 'payment_attempt' THEN cnt END), 0) as payment_attempt,
        COALESCE(SUM(CASE WHEN event_type = 'order_completed' THEN cnt END), 0) as order_completed
    FROM (
        SELECT date, 'add_to_cart' as event_type, COUNT(*) as cnt FROM events.add_to_cart GROUP BY date
        UNION ALL
        SELECT date, 'begin_checkout' as event_type, COUNT(*) as cnt FROM events.begin_checkout GROUP BY date
        UNION ALL
        SELECT date, 'checkout_step_view' as event_type, COUNT(*) as cnt FROM events.checkout_step_view GROUP BY date
        UNION ALL
        SELECT date, 'form_error' as event_type, COUNT(*) as cnt FROM events.form_error GROUP BY date
        UNION ALL
        SELECT date, 'payment_attempt' as event_type, COUNT(*) as cnt FROM events.payment_attempt GROUP BY date
        UNION ALL
        SELECT date, 'order_completed' as event_type, COUNT(*) as cnt FROM events.order_completed GROUP BY date
    ) subq
    GROUP BY date
    ORDER BY date
""").df()

print("Event Counts by Date:")
print(event_counts_by_date.to_string(index=False))


### 1.2 Variant Balance Check


In [ ]:
# Query add_to_cart counts by variant for the most recent date
variant_balance = conn.execute("""
    SELECT 
        variant,
        COUNT(DISTINCT user_id) as users,
        ROUND(COUNT(DISTINCT user_id) * 100.0 / SUM(COUNT(DISTINCT user_id)) OVER (), 1) as pct
    FROM events.add_to_cart
    WHERE date = (SELECT MAX(date) FROM events.add_to_cart)
    GROUP BY variant
    ORDER BY variant
""").df()

print("Add-to-Cart Users by Variant (Most Recent Date):")
print(variant_balance.to_string(index=False))


## 2. Funnel Step-Through Rates

Analyze conversion rates at each stage of the checkout funnel:
- Add to Cart → Begin Checkout
- Begin Checkout → Complete All Steps
- Step-level progression (address → shipping → payment → review)
- Payment Attempt → Order Completed


### 2.1 Overall Funnel Metrics


In [ ]:
# Overall funnel: adders → orders by variant
print("="*60)
print("OVERALL FUNNEL: Adders → Orders by Variant")
print("="*60)

overall_funnel = conn.execute("""
    WITH adders AS (
        SELECT variant, COUNT(DISTINCT user_id) as adders
        FROM marts.fct_experiments
        GROUP BY variant
    ),
    orders AS (
        SELECT variant, COUNT(DISTINCT user_id) as orderers
        FROM marts.fct_orders
        GROUP BY variant
    )
    SELECT 
        a.variant,
        a.adders,
        COALESCE(o.orderers, 0) as orderers,
        ROUND(COALESCE(o.orderers, 0) * 100.0 / a.adders, 1) as conversion_rate_pct
    FROM adders a
    LEFT JOIN orders o ON a.variant = o.variant
    ORDER BY a.variant
""").df()

print(overall_funnel.to_string(index=False))

# Step-level progression from checkout steps
print("\n")
print("="*60)
print("STEP-LEVEL PROGRESSION")
print("="*60)

step_progression = conn.execute("""
    SELECT 
        step_name,
        step_index,
        COUNT(DISTINCT checkout_id) as checkouts,
        ROUND(AVG(median_latency_ms), 0) as avg_median_latency_ms,
        ROUND(SUM(error_events) * 100.0 / COUNT(DISTINCT checkout_id), 1) as error_rate_pct
    FROM marts.fct_checkout_steps
    GROUP BY step_name, step_index
    ORDER BY step_index
""").df()

print(step_progression.to_string(index=False))


### 2.2 Step-by-Step Progression


### 2.3 Variant Comparison


## 3. Guardrail Metrics

Preview key guardrail metrics to ensure data quality and business logic:
- Payment authorization rate
- Order value distribution
- Form error rates


In [ ]:
# Guardrail Metrics for Most Recent Date
print("="*70)
print("GUARDRAIL METRICS BY VARIANT (Most Recent Date)")
print("="*70)

guardrails = conn.execute("""
    WITH most_recent_date AS (
        SELECT MAX(date) as max_date FROM events.add_to_cart
    ),
    payment_auth AS (
        SELECT 
            variant,
            COUNT(*) as total_attempts,
            SUM(CASE WHEN authorized THEN 1 ELSE 0 END) as authorized_attempts,
            ROUND(SUM(CASE WHEN authorized THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as auth_rate_pct
        FROM events.payment_attempt, most_recent_date
        WHERE date = most_recent_date.max_date
        GROUP BY variant
    ),
    order_values AS (
        SELECT 
            variant,
            COUNT(*) as total_orders,
            ROUND(AVG(order_value), 2) as avg_order_value
        FROM events.order_completed, most_recent_date
        WHERE date = most_recent_date.max_date
        GROUP BY variant
    )
    SELECT 
        p.variant,
        p.total_attempts,
        p.authorized_attempts,
        p.auth_rate_pct,
        o.total_orders,
        o.avg_order_value
    FROM payment_auth p
    JOIN order_values o ON p.variant = o.variant
    ORDER BY p.variant
""").df()

print(guardrails.to_string(index=False))


### 3.1 Payment Authorization Rate


### 3.2 Order Value Distribution


### 3.3 Form Error Rate


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Query funnel volumes per variant for most recent date
funnel_data = conn.execute("""
    WITH most_recent_date AS (
        SELECT MAX(date) as max_date FROM events.add_to_cart
    )
    SELECT 
        variant,
        COUNT(DISTINCT user_id) as adders,
        (SELECT COUNT(DISTINCT checkout_id) 
         FROM events.begin_checkout, most_recent_date 
         WHERE date = max_date AND begin_checkout.variant = a.variant) as begin_checkout,
        (SELECT COUNT(*) 
         FROM events.payment_attempt, most_recent_date 
         WHERE date = max_date AND payment_attempt.variant = a.variant) as payment_attempt,
        (SELECT COUNT(*) 
         FROM events.order_completed, most_recent_date 
         WHERE date = max_date AND order_completed.variant = a.variant) as orders
    FROM events.add_to_cart a, most_recent_date
    WHERE a.date = most_recent_date.max_date
    GROUP BY variant
    ORDER BY variant
""").df()

# Prepare data for grouped bar chart
stages = ['Adders', 'Begin Checkout', 'Payment Attempt', 'Orders']
control_values = funnel_data[funnel_data['variant'] == 'control'][['adders', 'begin_checkout', 'payment_attempt', 'orders']].values[0]
treatment_values = funnel_data[funnel_data['variant'] == 'treatment'][['adders', 'begin_checkout', 'payment_attempt', 'orders']].values[0]

x = np.arange(len(stages))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(x - width/2, control_values, width, label='Control')
ax.bar(x + width/2, treatment_values, width, label='Treatment')

ax.set_xlabel('Funnel Stage')
ax.set_ylabel('Count')
ax.set_title('Checkout Funnel Volumes by Variant (Most Recent Date)')
ax.set_xticks(x)
ax.set_xticklabels(stages)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


## 4. Visualizations

Create visual representations of key funnel metrics.


In [ ]:
# Query latency_ms for payment step on most recent date
latency_data = conn.execute("""
    SELECT latency_ms
    FROM events.checkout_step_view
    WHERE step_name = 'payment'
    AND date = (SELECT MAX(date) FROM events.add_to_cart)
""").df()

# Create histogram
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(latency_data['latency_ms'], bins=30, edgecolor='black')

ax.set_xlabel('Latency (ms)')
ax.set_ylabel('Frequency')
ax.set_title('Payment Step Latency Distribution (Most Recent Date)')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Display summary statistics
print(f"Payment Step Latency Statistics:")
print(f"  Count: {len(latency_data):,}")
print(f"  Mean: {latency_data['latency_ms'].mean():.1f} ms")
print(f"  Median: {latency_data['latency_ms'].median():.1f} ms")
print(f"  Min: {latency_data['latency_ms'].min()} ms")
print(f"  Max: {latency_data['latency_ms'].max()} ms")


In [ ]:
from pathlib import Path

# Create output directory if it doesn't exist
output_dir = Path("reports/figures")
output_dir.mkdir(parents=True, exist_ok=True)

# 1. Variant Funnel Summary (adders → orders by variant)
variant_funnel_summary = conn.execute("""
    WITH adders AS (
        SELECT variant, COUNT(DISTINCT user_id) as adders
        FROM marts.fct_experiments
        GROUP BY variant
    ),
    orders AS (
        SELECT variant, COUNT(DISTINCT user_id) as orderers
        FROM marts.fct_orders
        GROUP BY variant
    )
    SELECT 
        a.variant,
        a.adders,
        COALESCE(o.orderers, 0) as orderers,
        ROUND(COALESCE(o.orderers, 0) * 100.0 / a.adders, 1) as conversion_rate_pct
    FROM adders a
    LEFT JOIN orders o ON a.variant = o.variant
    ORDER BY a.variant
""").df()

# Save to CSV
funnel_path = output_dir / "variant_funnel_summary.csv"
variant_funnel_summary.to_csv(funnel_path, index=False)

# 2. Guardrails Preview (payment auth rate and avg order value)
guardrails_preview = conn.execute("""
    WITH most_recent_date AS (
        SELECT MAX(date) as max_date FROM events.add_to_cart
    ),
    payment_auth AS (
        SELECT 
            variant,
            COUNT(*) as total_attempts,
            SUM(CASE WHEN authorized THEN 1 ELSE 0 END) as authorized_attempts,
            ROUND(SUM(CASE WHEN authorized THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as auth_rate_pct
        FROM events.payment_attempt, most_recent_date
        WHERE date = most_recent_date.max_date
        GROUP BY variant
    ),
    order_values AS (
        SELECT 
            variant,
            COUNT(*) as total_orders,
            ROUND(AVG(order_value), 2) as avg_order_value
        FROM events.order_completed, most_recent_date
        WHERE date = most_recent_date.max_date
        GROUP BY variant
    )
    SELECT 
        p.variant,
        p.total_attempts,
        p.authorized_attempts,
        p.auth_rate_pct,
        o.total_orders,
        o.avg_order_value
    FROM payment_auth p
    JOIN order_values o ON p.variant = o.variant
    ORDER BY p.variant
""").df()

# Save to CSV
guardrails_path = output_dir / "guardrails_preview.csv"
guardrails_preview.to_csv(guardrails_path, index=False)

# Print output paths
print("✓ Results exported to CSV:")
print(f"  1. {funnel_path.resolve()}")
print(f"  2. {guardrails_path.resolve()}")


### 4.2 Latency Distribution

Show the distribution of page load times (latency_ms) across checkout steps.


---

## Summary & Next Steps


### 4.1 Funnel Bar Chart

Visualize drop-off at each stage of the checkout funnel, comparing control vs treatment.
